<a href="https://colab.research.google.com/github/CarlosM2002/MineriadeDatos/blob/main/Asignacion1_29551025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Asignación 1 - Preprocesamiento del Medical Student Dataset
# Carlos Monsalve - CI:29551025

![Portada](https://drive.google.com/uc?export=download&id=1LHlxhvFAaSjMtA7s07cwfMBVESpJ0m7S)



A continuación, realizaremos el preprocesamiento estructural y funcional del dataset con datos de estudiantes de medicina, el cual contiene 200.000 registros, por lo que, mediante técnicas de limpieza y transformación ajustaremos los valores para acondicionar los datos de forma adecuada para eliminar problemas de calidad y errores en su posterior uso en tareas de Minería de datos.

In [ ]:
import pandas as pd
import numpy as np

url = f'https://drive.google.com/uc?export=download&id=1iazg0c6462qSu1LNFsVyj05Iw9GAuhu6'
df = pd.read_csv(url)
print(df.shape)
df.head()


(200000, 13)


,Student ID,Age,Gender,Height,Weight,Blood Type,BMI,Temperature,Heart Rate,Blood Pressure,Cholesterol,Diabetes,Smoking
0,1.0,18.0,Female,161.777924,72.354947,O,27.645835,NaN,95.0,109.0,203.0,No,NaN
1,2.0,NaN,Male,152.069157,47.630941,B,NaN,98.714977,93.0,104.0,163.0,No,No
2,3.0,32.0,Female,182.537664,55.741083,A,16.729017,98.260293,76.0,130.0,216.0,Yes,No
3,NaN,30.0,Male,182.112867,63.332207,B,19.096042,98.839605,99.0,112.0,141.0,No,Yes
4,5.0,23.0,Female,NaN,46.234173,O,NaN,98.480008,95.0,NaN,231.0,No,No


## Preprocesamiento Estructural

Antes de realizar cualquier modificación en el dataset, se debe observar con detalle como esta conformado para verificar si cumple con los objetivos del preprocesamiento estructural. En este caso, se observó que el dataset tiene múltiples filas para el mismo Student ID, donde la información no solo está duplicada, sino también dispersa. Un ejemplo sería el Student_ID=6, que como podemos observar, tiene dos registros en donde en el registro 100005 se muestra si tiene diabetes pero no su peso y tamaño, mientras que en el registro 5 ocurre lo contrario. Esto rompe la regla fundamental de que cada observación debe corresponder a una sola fila.

In [ ]:
duplicados = df.duplicated(subset=['Student ID'], keep=False)
filas_duplicadas = df[duplicados]
filas_duplicadas = filas_duplicadas.sort_values(by='Student ID')
print(f"Se encontraron {len(filas_duplicadas)} filas correspondientes a IDs duplicados.")
display(filas_duplicadas.head(6))

Se encontraron 182048 filas correspondientes a IDs duplicados.


,Student ID,Age,Gender,Height,Weight,Blood Type,BMI,Temperature,Heart Rate,Blood Pressure,Cholesterol,Diabetes,Smoking
100001,2.0,34.0,Male,152.069157,47.630941,B,20.597139,98.714977,93.0,104.0,163.0,No,No
1,2.0,NaN,Male,152.069157,47.630941,B,NaN,98.714977,93.0,104.0,163.0,No,No
100004,5.0,23.0,Female,179.339293,46.234173,O,14.375143,98.480008,95.0,139.0,231.0,No,No
4,5.0,23.0,Female,NaN,46.234173,O,NaN,98.480008,95.0,NaN,231.0,No,No
5,6.0,32.0,NaN,151.491294,68.647805,B,29.912403,99.668373,70.0,128.0,183.0,NaN,Yes
100005,6.0,32.0,Female,NaN,NaN,B,29.912403,99.668373,70.0,128.0,183.0,No,Yes


Antes de solucionar este problema, validaremos si existen estudiantes sin un ID asignado para evitar perdida de información antes de unir aquellos estudiantes que SI tienen sus valores duplicados o particionados, por ello separamos en dos tablas los estudiantes que si tienen ID de los que no y aprovechamos para asignarle un ID a aquellos registros que no lo tienen.

Luego, en la tabla de los estudiantes con ID, reagrupamos las filas aplicando la función groupby y el método first para combinar las filas duplicadas ya que los datos no se contradicen entre si, sino que se complementan. por ejemplo para el caso del Student_Id=6, el problema se encuentra en la fragmentación de la información. Ya que en uno tiene si tiene diabetes pero no el tamaño y peso mientras que el otro ocurre lo contrario. Por lo que podemos unificarlo en una fila que contenga toda la información necesaria sin contradecir alguno de sus otros valores, de forma tal que cada observación está encapsulada en una fila de forma precisa.

In [ ]:
mask_nulos = df['Student ID'].isna()
df_sin_id = df[mask_nulos].copy()
print(f"Filas sin ID: {len(df_sin_id)}")
id_inicio = int(df['Student ID'].max() + 1)
df_sin_id['Student ID'] = range(id_inicio, id_inicio + len(df_sin_id))

df_con_id = df[df['Student ID'].notna()]
df_fixed_con_id = df_con_id.groupby('Student ID', as_index=False).first()
print(f"Filas con ID: {len(df_fixed_con_id)}")
df_final = pd.concat([df_fixed_con_id, df_sin_id], ignore_index=True)

ids_pruebas = [2.0, 5.0, 6.0]
print(f"\nSolución: Filas consolidadas para los IDs {ids_pruebas} ---")
display(df_final[df_final['Student ID'].isin(ids_pruebas)])

print(f"\nTotal de filas antes: {len(df)}")
print(f"Total de filas después: {len(df_final)}")

Filas sin ID: 20000
Filas con ID: 98976

Solución: Filas consolidadas para los IDs [2.0, 5.0, 6.0] ---


,Student ID,Age,Gender,Height,Weight,Blood Type,BMI,Temperature,Heart Rate,Blood Pressure,Cholesterol,Diabetes,Smoking
1,2.0,34.0,Male,152.069157,47.630941,B,20.597139,98.714977,93.0,104.0,163.0,No,No
4,5.0,23.0,Female,179.339293,46.234173,O,14.375143,98.480008,95.0,139.0,231.0,No,No
5,6.0,32.0,Female,151.491294,68.647805,B,29.912403,99.668373,70.0,128.0,183.0,No,Yes



Total de filas antes: 200000
Total de filas después: 118976


Además del problema inicial respecto a como una observación estaba representada en multiples filas, este dataset no tiene otros problemas con respecto a su preprocesamiento estructural, ya que las columnas contienen una única variable, las variables se presentan en filas y columnas y tampoco tiene descriptores como valores.



## Preprocesamiento Funcional

A continuación, viene el análisis sobre los problemas con respecto al preprocesamiento funcional, el cual, uno de sus mayores problemas fue solucionado parcialmente en el paso anterior al asignar un Student_ID a aquellos registros que lo tenían en nulo, sin embargo, esto se hizo solo para Student_ID y se debería hacer para todos los demás valores que puedan sesgar de forma errónea el algoritmo.

Para ello, separaremos las columnas en columnas númericas y columnas categóricas, rellenando los vacios de las numéricas con la mediana para evitar distorsionar los valores con extremos y los de las categóricas con el valor más frecuente, consolidando la integridad del dataset antes de ser modelado.

In [ ]:
cols = [c for c in df_final.columns if c != 'Student ID']
print(f"Total Valores Nulos:\n{df_final[cols].isnull().sum()}")

num_cols = df_final[cols].select_dtypes(include='number').columns
df_final[num_cols] = df_final[num_cols].fillna(df_final[num_cols].median())

cat_cols = df_final[cols].select_dtypes(exclude='number').columns
df_final[cat_cols] = df_final[cat_cols].fillna(df_final[cat_cols].mode().iloc[0])

print(f"Total Valores Nulos luego de insertar valores:\n{df_final[cols].isnull().sum()}")

df_final.head()

Total Valores Nulos:
Age               4615
Gender            4614
Height            4596
Weight            4530
Blood Type        4563
BMI               4623
Temperature       4712
Heart Rate        4674
Blood Pressure    4551
Cholesterol       4589
Diabetes          4629
Smoking           4619
dtype: int64
Total Valores Nulos luego de insertar valores:
Age               0
Gender            0
Height            0
Weight            0
Blood Type        0
BMI               0
Temperature       0
Heart Rate        0
Blood Pressure    0
Cholesterol       0
Diabetes          0
Smoking           0
dtype: int64


,Student ID,Age,Gender,Height,Weight,Blood Type,BMI,Temperature,Heart Rate,Blood Pressure,Cholesterol,Diabetes,Smoking
0,1.0,18.0,Female,161.777924,72.354947,O,27.645835,98.599486,95.0,109.0,203.0,No,No
1,2.0,34.0,Male,152.069157,47.630941,B,20.597139,98.714977,93.0,104.0,163.0,No,No
2,3.0,32.0,Female,182.537664,55.741083,A,16.729017,98.260293,76.0,130.0,216.0,Yes,No
3,4.0,30.0,Male,182.112867,63.332207,B,19.096042,98.839605,99.0,112.0,141.0,No,Yes
4,5.0,23.0,Female,179.339293,46.234173,O,14.375143,98.480008,95.0,139.0,231.0,No,No


Otro problema presente en el dataset se encuentras en las variables categóricas que se encuentran en formato de texto, la cuales son Género, Tipo de Sangre, Diabetes y Fumador, las cuales se almacenan en cadenas de texto, lo cual complica su procesamiento ya que la mayoría de modelos de ML requieren entradas numéricas para procesar la información de forma eficiente, por ello, vamos a aplicar primero para las variables de Diabetes y Fumador una codificación binaria, en donde el 0 representa el "No" y el 1 el "Yes" Por otro lado, para Blood Type y Gender la representaremos como arreglos n-dimensionales de forma tal que convertimos las cadenas de texto en valores numéricos.

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
cols_to_encode = ['Diabetes', 'Smoking']
for col in cols_to_encode:
    df_final[col] = le.fit_transform(df_final[col])

df_final = pd.get_dummies(df_final, columns=['Blood Type', 'Gender'], dtype=int)

df_final.head()



,Student ID,Age,Height,Weight,BMI,Temperature,Heart Rate,Blood Pressure,Cholesterol,Diabetes,Smoking,Blood Type_A,Blood Type_AB,Blood Type_B,Blood Type_O,Gender_Female,Gender_Male
0,1.0,18.0,161.777924,72.354947,27.645835,98.599486,95.0,109.0,203.0,0,0,0,0,0,1,1,0
1,2.0,34.0,152.069157,47.630941,20.597139,98.714977,93.0,104.0,163.0,0,0,0,0,1,0,0,1
2,3.0,32.0,182.537664,55.741083,16.729017,98.260293,76.0,130.0,216.0,1,0,1,0,0,0,1,0
3,4.0,30.0,182.112867,63.332207,19.096042,98.839605,99.0,112.0,141.0,0,1,0,0,1,0,0,1
4,5.0,23.0,179.339293,46.234173,14.375143,98.480008,95.0,139.0,231.0,0,0,0,0,0,1,1,0


Por último, vamos a convertir los valores de Student_ID y edad a variables de tipo entero, esto debido a que al estar en valor flotante complica y afecta de forma negativa el análisis de estas variables y eliminaremos la columna del BMI porque aunque el dato es útil, al eliminarlo reduce la dimensión del dataset y este valor se puede obtener a partir de altura y peso, por lo que no es necesario tenerla explicitamente en el dataset.

In [ ]:
df_final['Student ID'] = df_final['Student ID'].astype(int)
df_final['Age'] = df_final['Age'].astype(int)
df_final.drop(columns=['BMI'], inplace=True)
print(df_final.shape)
df_final.head()

(118976, 16)


,Student ID,Age,Height,Weight,Temperature,Heart Rate,Blood Pressure,Cholesterol,Diabetes,Smoking,Blood Type_A,Blood Type_AB,Blood Type_B,Blood Type_O,Gender_Female,Gender_Male
0,1,18,161.777924,72.354947,98.599486,95.0,109.0,203.0,0,0,0,0,0,1,1,0
1,2,34,152.069157,47.630941,98.714977,93.0,104.0,163.0,0,0,0,0,1,0,0,1
2,3,32,182.537664,55.741083,98.260293,76.0,130.0,216.0,1,0,1,0,0,0,1,0
3,4,30,182.112867,63.332207,98.839605,99.0,112.0,141.0,0,1,0,0,1,0,0,1
4,5,23,179.339293,46.234173,98.480008,95.0,139.0,231.0,0,0,0,0,0,1,1,0
